In [1]:
import os
import requests
import numpy as np
import pandas as pd


from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

# 01 Prepare data

- NHK program list text and genres (too few)
- MARC-ja (not available)
- JCoLA https://github.com/osekilab/JCoLA <-- we are using this

In [2]:
# def get_nhk_programs(url: str) -> dict:
#     """
#     NHK番組表を取得する
#     """

#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#         data = response.json()
#     except:
#         data = None
#     return data


# def cast_json_to_pandas(json_data):
#     """
#     JSONデータをPandasのDataFrameに変換する
#     """
#     if json_data is None:
#         return None
#     return pd.json_normalize(json_data["list"]["g1"])

In [3]:
# # https://api.nhk.or.jp/v2/pg/genre/{area}/{service}/{genre}/{date}.json?key={apikey}

# apikey = os.environ.get("NHK_API_KEY", None)
# area = "130"  # 東京
# service = "g1"  # NHK総合
# start_date = "2024-04-24"  # 開始日
# end_date = "2024-11-24"  # 終了日

# get_list_of_dates = (
#     lambda start_date, end_date: pd.date_range(start_date, end_date)
#     .strftime("%Y-%m-%d")
#     .tolist()
# )

# dates = get_list_of_dates(start_date, end_date)

# nhk_programs_json = map(
#     lambda date: get_nhk_programs(
#         f"https://api.nhk.or.jp/v2/pg/list/{area}/{service}/{date}.json?key={apikey}"
#     ),
#     dates,
# )
# nhk_programs_json = filter(lambda x: x is not None, nhk_programs_json)
# df = pd.concat(map(cast_json_to_pandas, nhk_programs_json))

In [4]:
# data_path = "../local/data/JCoLA-main/data/jcola-v1.0/in_domain_train-v1.0.tsv"
# df = pd.read_csv(data_path, sep="\t")


# 02 Featrue engineering

In [5]:
# os.makedirs("../local/data", exist_ok=True)
# df.to_csv("../local/data/nhk_programs.csv", index=False)
# df = pd.read_csv("../local/data/nhk_programs.csv", dtype=str).dropna()

data_path = "../local/data/JCoLA-main/data/jcola-v1.0/in_domain_train-v1.0.tsv"
df = pd.read_csv(data_path, sep="\t")

# Text feature extraction
model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite"
# model_name = "studio-ousia/luke-japanese-base-lite"
encoder = HuggingFaceEmbeddings(model_name=model_name)

# text_features = ["title", "subtitle", "content", "act"]
text_features = ["sentence"]

for i in text_features:
    embeddings = pd.DataFrame(
        encoder.embed_documents(df[i]), index=df.index
    ).add_prefix(f"embed_{i}_")
    df = df.join(embeddings, how="left")


No sentence-transformers model found with name studio-ousia/luke-japanese-base-lite. Creating a new one with MEAN pooling.


config.json:   0%|          | 0.00/873 [00:00<?, ?B/s]

c:\Users\ray\Desktop\src\leetcode\.venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ray\.cache\huggingface\hub\models--studio-ousia--luke-japanese-base-lite. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/536M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/842k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

In [6]:
# Choose training features
# training_features = [i for i in df.columns if "embed_" in i]
# df = df[training_features + ["label"]]

# Target label data clearning
# df["genres"] = df["genres"].map(eval)
# df = df.explode("genres").reset_index(drop=True)

# Choose training features
training_features = [i for i in df.columns if "embed_" in i]
target_label = "label"
df = df[training_features + [target_label]]
df.head(1)

,embed_sentence_0,embed_sentence_1,embed_sentence_2,embed_sentence_3,embed_sentence_4,embed_sentence_5,embed_sentence_6,embed_sentence_7,embed_sentence_8,embed_sentence_9,...,embed_sentence_759,embed_sentence_760,embed_sentence_761,embed_sentence_762,embed_sentence_763,embed_sentence_764,embed_sentence_765,embed_sentence_766,embed_sentence_767,label
0,0.167172,-0.194901,-0.043595,-0.167724,-0.325008,-0.127364,0.030671,0.122342,-0.002697,-0.281892,...,-0.1669,0.089946,0.121289,-0.060072,-0.404347,0.070873,0.015921,0.176191,-0.096621,1


In [7]:


# train classifier by logitic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df[training_features]
y = df[target_label]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("train score ", clf.score(X_train, y_train))
print("test score ", clf.score(X_test, y_test))
print(classification_report(y_test, y_pred))


train score  0.8599819331526649
test score  0.8258670520231214
              precision    recall  f1-score   support

           0       0.46      0.14      0.21       236
           1       0.85      0.97      0.90      1148

    accuracy                           0.83      1384
   macro avg       0.66      0.55      0.56      1384
weighted avg       0.78      0.83      0.78      1384



In [8]:
# train text classifier with light boost
import lightgbm as lgb
from lightgbm import LGBMClassifier

clf = LGBMClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)

print("train score ", clf.score(X_train, y_train))
print("test score ", clf.score(X_test, y_test))

print(classification_report(y_test, y_pred))

 


c:\Users\ray\Desktop\src\leetcode\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\ray\Desktop\src\leetcode\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Number of positive: 4621, number of negative: 914
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 5535, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.834869 -> initscore=1.620536
[LightGBM] [Info] Start training from score 1.620536
train score  1.0
test score  0.8323699421965318
              precision    recall  f1-score   support

           0       0.54      0.11      0.19       236
           1       0.84      0.98      0.91      1148

    accuracy                           0.83      1384
   macro avg       0.69      0.55      0.55      1384
weighted avg       0.79      0.83      0.78      1384



In [9]:
# Using HuggingFace api for feature extraction

# model_id = "oshizo/sbert-jsnli-luke-japanese-base-lite"
# hf_token = os.environ.get("HUGGINGFACE_API_KEY_READ", None)
# api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
# headers = {"Authorization": f"Bearer {hf_token}"}

# def encode_texts(texts):
#     """
#     テキストをエンコードする
#     """
#     response = requests.post(
#         api_url,
#         headers=headers,
#         json={"inputs": texts, "options": {"wait_for_model": True}},
#     )
#     return response.json()

# tmp = pd.read_csv("../local/data/nhk_programs.csv", dtype=str)
# tmp = tmp.dropna()['title'].tolist()
# tmp = encode_texts(tmp)